This notebooks creates a new network specifically designed for node2vec usage later

In [1]:
from pyspark.sql import SparkSession
import sparknlp
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.sql import types as T
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
import itertools
from pyspark.sql.functions import col, when, least, greatest, lit

spark = SparkSession \
        .builder \
        .appName("network") \
        .getOrCreate()

24/11/27 17:57:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("../cleaned_moral_scores.csv", header= True).select('id', 'cleaned_text', 'emo_pos', 'emo_neg', 
                                                                          'emo_anx','emo_anger','emo_sad', 'moral')

In [3]:
data.show(10)

+-----+--------------------+-------+-------+-------+---------+-------+-----+
|   id|        cleaned_text|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|
+-----+--------------------+-------+-------+-------+---------+-------+-----+
|hk5r2|i had an appointm...|    0.0|    0.0|    0.0|      0.0|    0.0|  0.0|
|iqimz|i created this si...|   2.56|    0.0|    0.0|      0.0|    0.0| 1.71|
|pfzt5|hello everyone  i...|   2.06|    0.0|    0.0|      0.0|    0.0| 0.52|
|pk714|i grew up with bo...|   1.71|    1.2|   0.34|      0.0|   0.51| 0.68|
|q0q8x|i have to ask whe...|   1.25|   1.61|   0.18|     0.18|    0.9| 0.18|
|q412v|nothing but oppor...|   1.05|   3.16|    0.0|      0.0|   3.16|  0.0|
|q5mqk|im getting out of...|   3.27|   1.96|   1.31|      0.0|    0.0|  0.0|
|q70xe|hey everyone firs...|    0.0|   1.96|    0.0|      0.0|    0.0|  0.0|
|q7mrn|facebook is great...|   0.96|    0.0|    0.0|      0.0|    0.0|  0.0|
|qcsyp|okay so im 18 yea...|   0.74|   0.74|    0.0|      0.0|    0.0|  0.0|

Data preprocessing

In [4]:
#Define stopwords
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "even", "one", "bye", "take", "wasnt"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year", "now"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement", "blah"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_", "ampxb"]

stopwords = english + time + reddit + topic_specific

In [5]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized')

lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("normalized")\
      .setOutputCol("lemmatized")

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('words') \
     .setStopWords(stopwords)

from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['words'])

my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          lemmatizer,
          stopwords_cleaner,
          finisher
      ])

In [6]:
pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.persist()
processed_data.show(10)

+-----+--------------------+-------+-------+-------+---------+-------+-----+--------------------+
|   id|        cleaned_text|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|      finished_words|
+-----+--------------------+-------+-------+-------+---------+-------+-----+--------------------+
|hk5r2|i had an appointm...|    0.0|    0.0|    0.0|      0.0|    0.0|  0.0|[appointment, den...|
|iqimz|i created this si...|   2.56|    0.0|    0.0|      0.0|    0.0| 1.71|[create, site, se...|
|pfzt5|hello everyone  i...|   2.06|    0.0|    0.0|      0.0|    0.0| 0.52|[recently, made, ...|
|pk714|i grew up with bo...|   1.71|    1.2|   0.34|      0.0|   0.51| 0.68|[grow, body, dysm...|
|q0q8x|i have to ask whe...|   1.25|   1.61|   0.18|     0.18|    0.9| 0.18|[content, never, ...|
|q412v|nothing but oppor...|   1.05|   3.16|    0.0|      0.0|   3.16|  0.0|[butt, opportunit...|
|q5mqk|im getting out of...|   3.27|   1.96|   1.31|      0.0|    0.0|  0.0|[comfort, zone, t...|
|q70xe|hey everyone 

In [7]:
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_words', outputCol='tf_features', minDF=0.01, vocabSize=1000)
tf_model = tfizer.fit(processed_data)
tf_result = tf_model.transform(processed_data)
vocabulary = tf_model.vocabulary


idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

processed_data.unpersist()
tfidf_result.persist()

DataFrame[id: string, cleaned_text: string, emo_pos: string, emo_neg: string, emo_anx: string, emo_anger: string, emo_sad: string, moral: string, finished_words: array<string>, tf_features: vector, tf_idf_features: vector]

In [8]:
# Define a function to filter words by their TF-IDF score
# UDF to map indices to words using the vocabulary
def filter_tfidf(features, threshold=10, vocabulary=None):
    if features is not None:
        # Filter based on TF-IDF score and map indices to actual words
        return [vocabulary[features.indices[i]] for i in range(len(features.values)) if features.values[i] >= threshold]
    return []

# Register the UDF
filter_udf = udf(lambda features: filter_tfidf(features, threshold=10, vocabulary=vocabulary), ArrayType(StringType()))

# Apply the filtering function
df_filtered_tfidf = tfidf_result.withColumn("filtered_words_tfidf", filter_udf("tf_idf_features"))

df_filtered_tfidf.show()
tfidf_result.unpersist()

+-----+--------------------+-------+-------+-------+---------+-------+-----+--------------------+--------------------+--------------------+--------------------+
|   id|        cleaned_text|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|      finished_words|         tf_features|     tf_idf_features|filtered_words_tfidf|
+-----+--------------------+-------+-------+-------+---------+-------+-----+--------------------+--------------------+--------------------+--------------------+
|hk5r2|i had an appointm...|    0.0|    0.0|    0.0|      0.0|    0.0|  0.0|[appointment, den...|(803,[13,36,43,76...|(803,[13,36,43,76...|                  []|
|iqimz|i created this si...|   2.56|    0.0|    0.0|      0.0|    0.0| 1.71|[create, site, se...|(803,[0,3,57,134,...|(803,[0,3,57,134,...|                  []|
|pfzt5|hello everyone  i...|   2.06|    0.0|    0.0|      0.0|    0.0| 0.52|[recently, made, ...|(803,[6,9,11,19,2...|(803,[6,9,11,19,2...|                  []|
|pk714|i grew up with bo...|   1.7

DataFrame[id: string, cleaned_text: string, emo_pos: string, emo_neg: string, emo_anx: string, emo_anger: string, emo_sad: string, moral: string, finished_words: array<string>, tf_features: vector, tf_idf_features: vector]

In [9]:
import itertools
def generate_edges(tokens):
    return [list(pair) for pair in itertools.combinations(tokens, 2)]

generate_edges_udf = udf(generate_edges, ArrayType(ArrayType(StringType())))

In [10]:
df_edges = df_filtered_tfidf.withColumn("edges", generate_edges_udf(F.col("filtered_words_tfidf")))
df_edges.show(10)

+-----+--------------------+-------+-------+-------+---------+-------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+
|   id|        cleaned_text|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|      finished_words|         tf_features|     tf_idf_features|filtered_words_tfidf|            edges|
+-----+--------------------+-------+-------+-------+---------+-------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+
|hk5r2|i had an appointm...|    0.0|    0.0|    0.0|      0.0|    0.0|  0.0|[appointment, den...|(803,[13,36,43,76...|(803,[13,36,43,76...|                  []|               []|
|iqimz|i created this si...|   2.56|    0.0|    0.0|      0.0|    0.0| 1.71|[create, site, se...|(803,[0,3,57,134,...|(803,[0,3,57,134,...|                  []|               []|
|pfzt5|hello everyone  i...|   2.06|    0.0|    0.0|      0.0|    0.0| 0.52|[recently, made, ...|(803,[6,

In [11]:
df_explode = df_edges.select(
    F.col("id"),
    F.explode(F.col("edges")).alias("edge"), F.col('emo_pos'),
    F.col('emo_neg'), F.col('emo_anx'), F.col('emo_anger'), 
    F.col('emo_sad'), F.col('moral'))

df_explode.show(10)

+-----+---------------+-------+-------+-------+---------+-------+-----+
|   id|           edge|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|
+-----+---------------+-------+-------+-------+---------+-------+-----+
|pk714|[wear, clothes]|   1.71|    1.2|   0.34|      0.0|   0.51| 0.68|
|q0q8x|   [water, sad]|   1.25|   1.61|   0.18|     0.18|    0.9| 0.18|
|rvjcf|   [read, play]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|  [read, drink]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf| [read, amount]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   [read, lift]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|  [play, drink]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf| [play, amount]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   [play, lift]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|[drink, amount]|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
+-----+---------------+-------+-------+-------+---------+-------

Create edges df

In [12]:
edges_df = df_explode.select(
    F.col("edge")[0].alias("node1"),
    F.col("edge")[1].alias("node2"))

edges_df = edges_df.withColumn("weight", lit(1))

In [13]:
# Normalize the pairs: ensure node1 is always less than node2, so they can be always on the same order
edges_df = edges_df.withColumn("node1_norm", least(col("node1"), col("node2"))) \
             .withColumn("node2_norm", greatest(col("node1"), col("node2"))) \
             .select('node1_norm', 'node2_norm', 'weight')

In [14]:
edges_df = edges_df.groupBy("node1_norm", "node2_norm").sum("weight") \
                        .withColumnRenamed("sum(weight)", "weight")

Create nodes df

In [15]:
nodes_df = df_explode.select(
    F.col("id"),
    F.explode(F.col("edge")).alias("node"), F.col('emo_pos'),
    F.col('emo_neg'), F.col('emo_anx'), F.col('emo_anger'), 
    F.col('emo_sad'), F.col('moral'))

nodes_df.show(20) 

nodes_df_g = nodes_df.groupBy("id", "node").agg(
    F.first('emo_pos').alias('emo_pos'),
    F.first('emo_neg').alias('emo_neg'),
    F.first('emo_anx').alias('emo_anx'),
    F.first('emo_anger').alias('emo_anger'),
    F.first('emo_sad').alias('emo_sad'),
    F.first('moral').alias('moral'))

nodes_df_g.show(20)

+-----+-------+-------+-------+-------+---------+-------+-----+
|   id|   node|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|
+-----+-------+-------+-------+-------+---------+-------+-----+
|pk714|   wear|   1.71|    1.2|   0.34|      0.0|   0.51| 0.68|
|pk714|clothes|   1.71|    1.2|   0.34|      0.0|   0.51| 0.68|
|q0q8x|  water|   1.25|   1.61|   0.18|     0.18|    0.9| 0.18|
|q0q8x|    sad|   1.25|   1.61|   0.18|     0.18|    0.9| 0.18|
|rvjcf|   read|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   play|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   read|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|  drink|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   read|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf| amount|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   read|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   lift|   2.34|   0.43|    0.0|     0.43|    0.0| 0.21|
|rvjcf|   play|   2.34|   0.43|    0.0| 

+-------+------------+-------+-------+-------+---------+-------+-----+
|     id|        node|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|
+-------+------------+-------+-------+-------+---------+-------+-----+
|1002lda|       grade|   0.82|   2.87|    0.0|      0.0|   2.05| 0.41|
|1002wwt|      always|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|        hand|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|  meditation|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|        meet|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|      mental|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|   recommend|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|      slowly|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1002wwt|       sport|   1.16|   1.01|   0.24|     0.07|    0.1| 0.17|
|1004pti|        meal|   0.27|   0.27|   0.13|      0.0|   0.13| 0.13|
|1004pti|        wake|   0.27|   0.27|   0.13|      0.0|   0.13| 0.13|
|1006c

Now, aggregate all words and average their emotions and morality scores for all documents in which they appear

In [16]:
final_nodes = nodes_df_g.groupBy("node").agg(
    F.avg('emo_pos').alias('emo_pos'),
    F.avg('emo_neg').alias('emo_neg'),
    F.avg('emo_anx').alias('emo_anx'),
    F.avg('emo_anger').alias('emo_anger'),
    F.avg('emo_sad').alias('emo_sad'),
    F.avg('moral').alias('moral'))

In [17]:
final_nodes.show(20)

+------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|        node|           emo_pos|           emo_neg|            emo_anx|          emo_anger|            emo_sad|              moral|
+------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|      online|0.8537845303867402|0.9379005524861878|0.25859116022099443|0.18361878453038674|0.16116022099447513|0.27157458563535913|
|   recognize|1.0374045801526717| 1.114885496183206| 0.3461068702290076|0.19709923664122134|0.18145038167938932|0.45038167938931295|
|       inner|1.1068803418803421|1.1653418803418805|0.30491452991452994|0.19931623931623932|  0.166025641025641|0.46333333333333326|
|      travel|              1.21|0.8630107526881721|0.31268817204301075|0.11193548387096774|0.15526881720430105| 0.2249462365591398|
|        hope|2.4231147540983606| 1.149344262295082| 0.26245901639344

Write to CSV files

In [18]:
final_nodes.write.mode("overwrite").csv("nodes_network3", header=True)

In [19]:
edges_df.write.mode("overwrite").csv("edges_network3", header=True)